In [1]:
import pandas as pd
import torch

/home/data/xuyijie/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Split Train and Test

In [3]:
df = pd.read_csv('/home/data/xuyijie/Documents/AllsidesScraper/news-title-bias/data/dataset/dataset.csv', header=0, encoding='utf_8_sig')
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=42)
train.to_csv('/home/data/xuyijie/Documents/AllsidesScraper/news-title-bias/data/dataset/train.csv', index=False, encoding='utf-8')
test.to_csv('/home/data/xuyijie/Documents/AllsidesScraper/news-title-bias/data/dataset/test.csv', index=False, encoding='utf-8')

# Load Datasets

In [32]:
from datasets import load_dataset
dataset = load_dataset('text', data_files={'train': '../data/dataset/train.csv', 'test': '../data/dataset/test.csv'})

Using custom data configuration default-463d696f02a7192d


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /home/yijiexu/.cache/huggingface/datasets/text/default-463d696f02a7192d/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 214855
    })
    test: Dataset({
        features: ['text'],
        num_rows: 23893
    })
})

# Tokenize

In [40]:
def tokenize_function(examples):
    labels = []
    texts = []
    for example in examples['text']:
        split = example.split(',', maxsplit=1)
        print(split[1])
        labels.append(int(split[1]))
        texts.append(split[0])
    tokenized = tokenizer(texts, padding='max_length', truncation=True, max_length=32)
    tokenized['labels'] = labels
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

  0%|          | 0/215 [00:00<?, ?ba/s]

-2
2
1
-2
2
-1
-1
2
-1
-2
1
-1
-1
2
0
1
2
0


IndexError: list index out of range

In [43]:
dataset['train']['text'][18]

'"Benjamin Netanyahu officially informed Israels President that he has successfully formed a new government some six weeks after the Knesset elections'

In [2]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file, header=0)
                                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        sample = {'text': row[0], 'label': row[1]}
        return sample

dataset = CustomDataset("../data/dataset/dataset.csv")

In [46]:
import psutil
import humanize
import os
import GPUtil as GPU

import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold 
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

#choose the same seed to assure that our model will be roproducible

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(2)

In [47]:
import pandas as pd
#We consider that our data is a csv file (2 columns : text and label)
#using pandas function (read_csv) to read the file
train=pd.read_csv('../data/dataset/dataset.csv', encoding='utf_8_sig', header=0)

feat_cols = "text"

In [5]:
train.label.unique()

array([ 0, -1,  2, -2,  1])

In [48]:
label_cols = ['Lean Left', 'Lean Right', 'Center', 'Left', 'Right']
train['label']=train.label.astype('category')

In [49]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [50]:
Y = to_categorical(train.label.astype('category'), num_classes=5)

In [51]:
l = [-2,-1,0,1,2]
for i in range(len(label_cols)) :     
     train[l[i]] = Y[:,i]

In [10]:
train

,text,label,-2,-1,0,1,2
0,Summary U.S. announces new $1.85 bln military aid package for Kyiv Zelenskiy's first trip abroad since Russian invasion began U.S. Patriot air defence battery part of new assistance Putin backs...,0,1,0,0,0,0
1,"In an extraordinary face-to-face meeting at the White House on Wednesday, President Joe Biden told Ukrainian President Volodymyr Zelenskyy that the U.S. ""will stay with you as long as it takes"" as...",-1,0,0,0,0,1
2,Ukrainian President Volodymyr Zelenskyy on Wednesday thanked the U.S. for its support for his nation and said he wants Americans to keep working together to defeat Russia's aspirations to control ...,2,0,0,1,0,0
3,"A patient is turned away from the emergency room due to full capacity at the Baoding No. 2 Central Hospital in Zhuozhou city in northern China's Hebei province on Wednesday, Dec. 21, 2022. China o...",-1,0,0,0,0,1
4,"Washington CNN — Two Americans who had been detained by the Taliban in Afghanistan have been released and are en route to Qatar, three sources familiar with the matter tell CNN. Two of the sourc...",-2,0,0,0,1,0
...,...,...,...,...,...,...,...
233565,"Comment on this story Comment Gift Article Share Hiring surged and wages grew more than they have in almost a decade, the government said Friday in a report seized on by Republicans just before t...",-1,0,0,0,0,1
233566,"Early withdrawals from a 401(k) typically come with a 10 percent tax. Under the new proposal, a person would be able to make one penalty-free withdrawal for unexpected or immediate expenses arisin...",-1,0,0,0,0,1
233567,"Listen Comment on this story Comment Gift Article Share The Senate on Tuesday took the first formal step toward advancing a bipartisan, roughly $1.7 trillion deal to fund the U.S. government, as ...",-1,0,0,0,0,1
233568,"Sen. Elizabeth Warren (D-Mass.) said Federal Reserve chair Jerome Powell is ""pushing hard to get more people fired because he thinks that is one way to help bring down inflation."" Pool via Getty I...",-2,0,0,0,1,0


In [52]:
err = []
y_pred_tot = []

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1997)
i = 1
for train_index, test_index in fold.split(train, train["label"]):
    train1_trn, train1_val = train.iloc[train_index], train.iloc[test_index]
    model = ClassificationModel(
        "roberta",
        "roberta-base",
        use_cuda=True,
        num_labels=4,
        args={
            "train_batch_size": 16,
            "reprocess_input_data": True,
            "overwrite_output_dir": True,
            "fp16": False,
            "do_lower_case": False,
            "num_train_epochs": 4,
            "max_seq_length": 128,
            "regression": False,
            "manual_seed": 1997,
            "learning_rate": 2e-5,
            "weight_decay": 0,
            "save_eval_checkpoints": True,
            "save_model_every_epoch": False,
            "silent": True,
        },
    )
    model.train_model(train1_trn)
    raw_outputs_val = model.eval_model(train1_val)[1]
    raw_outputs_vals = softmax(raw_outputs_val, axis=1)
    print(f"Log_Loss: {log_loss(train1_val['label'], raw_outputs_vals)}")
    err.append(log_loss(train1_val["label"], raw_outputs_vals))


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.